In [ ]:
## This is a sample code of 2017 HAB composite image creation to calculate Annual Weekly or Bi-Weekly MAX 

import os
import numpy as np
import rasterio

# Define the root directory containing the raster datasets
root_dir = "H:\\ALS\\Cyano_Bacteria\\2017"

# Traverse through each subdirectory within the root directory
for subdir, dirs, files in os.walk(root_dir):
    image_list = []  # List to store raster data arrays
    
    # Loop through each file in the current subdirectory
    for file in files:
        # Identify raster files with the prefix 'extracted_' and '.tif' extension
        if file.startswith("2017_") and file.endswith((".tif", ".TIF")):
            file_path = os.path.join(subdir, file)
            
            # Open the raster file and read it as a masked array to handle no-data values
            with rasterio.open(file_path) as src:
                image_list.append(src.read(masked=True))
    
    # If images are found in the list, proceed to create a composite image
    if image_list:
        # Calculate the maximum value for each pixel across all images
        composite_array = np.ma.max(np.ma.stack(image_list, axis=0), axis=0)
        
        # Retrieve metadata from the first raster file for use in the composite image
        with rasterio.open(os.path.join(subdir, files[0])) as src:
            meta = src.meta
        
        # Update the metadata to reflect the number of layers in the composite image
        meta.update(count=composite_array.shape[0])
        
        # Define the output path and filename for the composite image
        folder_name = os.path.basename(subdir)
        composite_path = os.path.join(subdir, f"{folder_name}_composite.tif")
        
        # Save the composite image to the specified location
        with rasterio.open(composite_path, 'w', **meta) as dst:
            dst.write(composite_array)
        
        # Print confirmation that the composite image has been saved
        print(f"Composite image saved: {composite_path}")
